In [1]:
# import the package 
import cobra

import pandas as pd
import numpy as np

pd.options.display.max_rows = 4000

from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [2]:
ipfa = cobra.io.read_sbml_model("iPFA/ipfasbml.xml")

Model does not contain SBML fbc package information.
'' is not a valid SBML 'SId'.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00001_c "H2O">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00002_c "ATP">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00003_c "NAD+">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00004_c "NADH">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00005_c "NADPH">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00006_c "NADP+">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Speci

In [3]:
ipfal21 = cobra.io.load_json_model("FBA_Carey_models/iPfal21.json")

In [4]:
print(ipfa.compartments)
print(ipfal21.compartments)

{'c': 'Cytoplasm', 'a': 'Apicoplast', 'r': 'Endoplasmic_reticulum', 'm': 'Mitochondrion', 'n': 'Nucleus', 'e': 'Extracellular'}
{'cytoplasm': 'c', 'extracellular': 'e', 'apicoplast': 'ap', 'mitochondrion': 'm', 'food_vacuole': 'fv'}


In [155]:
# find all substrates of the reaction
biomets = ipfa.reactions.Biomass_rxn_c.metabolites #r_4041 = biomassreaction 8

s = list(biomets.values()) #stoichiometry of the metabolites in the reaction
m = list(biomets.keys()) # ID of the metabolite in the reaction

# print a table of metabolites and their stoichiometry

metname = [mi.name for mi in m]# names of the metabolite in the reaction

# table printing (this is very messy)

biomass = pd.DataFrame.from_records(map(list, zip(*[m,metname,s])),columns=['ID', 'metabolite','stoichiometry'])
biomass.sort_values(by='metabolite', ascending='False')

,ID,metabolite,stoichiometry
64,C00712_c,(9Z)-Octadecenoic acid,-0.001
51,C00641_c,"1,2-Diacyl-sn-glycerol",-0.003
43,C00234_c,10-Formyltetrahydrofolate,-0.015
38,C00143_c,"5,10-Methylenetetrahydrofolate",-0.015
69,C00440_c,5-Methyltetrahydrofolate,-0.015
74,C00008_c,ADP,21.360
1,C00002_c,ATP,-21.406
77,Biomass_c,Biomass,1.000
68,C00120_c,Biotin,-0.008
21,C00063_c,CTP,-0.013


In [165]:
# find all substrates of the biomass reaction
biomets = ipfal21.reactions.biomass.metabolites 

s = list(biomets.values()) #stoichiometry of the metabolites in the reaction
m = list(biomets.keys()) # ID of the metabolite in the reaction

# print a table of metabolites and their stoichiometry

metname = [mi.name for mi in m]# names of the metabolite in the reaction

# table printing (this is very messy)

biomass_ipfal = pd.DataFrame.from_records(map(list, zip(*[m,metname,s])),columns=['ID', 'metabolite','stoichiometry'])
biomass_ipfal.sort_values(by='metabolite', ascending='True')

,ID,metabolite,stoichiometry
0,10fthf_c,10_Formyltetrahydrofolate,-0.000223
1,2ohph_m,2_Octaprenyl_6_hydroxyphenol,-0.000223
3,akg_c,2_Oxoglutarate,-0.010000
39,mlthf_c,5_10_Methylenetetrahydrofolate,-0.000223
54,thf_c,5_6_7_8_Tetrahydrofolate,-0.000223
2,adp_c,ADP,59.810000
9,atp_c,ATP,-60.010000
42,nh4_c,Ammonium,-0.011843
14,ctp_c,CTP,-0.043700
13,coa_c,Coenzyme_A,-0.000576


In [160]:
#biomass_ipfal.metabolite[biomass_ipfal.metabolite.isin(biomass.metabolite)]
kegg_ids = []
ids = []
for i,met in enumerate(m):
    try:
        kegg = met.annotation['kegg.compound']
    except:
        kegg=''
    kegg_ids.extend(kegg)
    ids.extend([i for _ in range(len(kegg))])


In [161]:
ipfa_kegg = [m.id[:-2] for m in biomass.ID]
biomass['KEGG']=ipfa_kegg
print(len(biomass))
print(len(biomass_ipfal))
print(len(biomass[biomass.KEGG.isin(kegg_ids)]))

not_venn_biomass=biomass[~biomass.KEGG.isin(kegg_ids)].sort_values(by='metabolite', ascending='True')
not_venn_biomass

78
61
49


,ID,metabolite,stoichiometry,KEGG
64,C00712_c,(9Z)-Octadecenoic acid,-0.001,C00712
51,C00641_c,"1,2-Diacyl-sn-glycerol",-0.003,C00641
69,C00440_c,5-Methyltetrahydrofolate,-0.015,C00440
77,Biomass_c,Biomass,1.000,Biomass
68,C00120_c,Biotin,-0.008,C00120
53,C05980_c,Cardiolipin,-0.008,C05980
62,C00187_c,Cholesterol,-0.046,C00187
52,C01245_c,"D-myo-Inositol 1,4,5-trisphosphate",-0.004,C01245
19,C00061_c,FMN,-0.013,C00061
47,C00325_c,GDP-L-fucose,-0.007,C00325


In [163]:
inter, ids_l, ids_i = np.intersect1d(np.asarray(kegg_ids), np.asarray(ipfa_kegg), return_indices=True)
not_included = np.asarray(ids)[~np.isin(np.arange(len(ids)), ids_l)]
venn_ipfal = biomass_ipfal.iloc[np.asarray(ids)[ids_l]].sort_values(by='metabolite', ascending='True')
venn_ipfal

,ID,metabolite,stoichiometry
0,10fthf_c,10_Formyltetrahydrofolate,-0.000223
39,mlthf_c,5_10_Methylenetetrahydrofolate,-0.000223
54,thf_c,5_6_7_8_Tetrahydrofolate,-0.000223
2,adp_c,ADP,59.810000
9,atp_c,ATP,-60.010000
42,nh4_c,Ammonium,-0.011843
14,ctp_c,CTP,-0.043700
13,coa_c,Coenzyme_A,-0.000576
46,ppi_c,Diphosphate,0.773900
20,fad_c,Flavin_adenine_dinucleotide_oxidized,-0.000223


In [164]:
biomass_ipfal[~biomass_ipfal.index.isin(venn_ipfal.index)].sort_values(by='metabolite', ascending='True')

,ID,metabolite,stoichiometry
1,2ohph_m,2_Octaprenyl_6_hydroxyphenol,-0.000223
3,akg_c,2_Oxoglutarate,-0.010000
21,fe2_c,Fe2,-0.007106
22,fe3_c,Fe3,-0.007106
37,mal__L_c,L_Malate,-0.010000
26,gthox_c,Oxidized_glutathione,-0.003000
50,ribflv_c,Riboflavin,-0.000223
10,biomass_c,biomass[c],1.000000
31,hemozoin_e,hemozoin,-0.100000
34,lac__L_e,lac_L[e],-0.010000


In [127]:
pepe_ex = ipfa.exchanges
df_ex_pepe = pd.DataFrame(pepe_ex, columns=['Reaction'])
df_ex_pepe['Metabolites'] = [list(df_ex_pepe.iloc[x][0].metabolites.keys())[0] for x in range(len(df_ex_pepe))]
df_ex_pepe['Bounds'] = [df_ex_pepe.iloc[x][0].bounds for x in range(len(df_ex_pepe))]


In [128]:
carey_ex = ipfal21.exchanges
df_ex_carey = pd.DataFrame(carey_ex, columns=['Reaction'])
df_ex_carey['Metabolites'] = [list(df_ex_carey.iloc[x][0].metabolites.keys())[0] for x in range(len(df_ex_carey))]
df_ex_carey['Bounds'] = [df_ex_carey.iloc[x][0].bounds for x in range(len(df_ex_carey))]


In [179]:
pepe_ex_mets = [list(pepe_ex[x].metabolites.keys())[0].id[:-2] for x in range(len(pepe_ex))]
pepe_ex_ids = [list(pepe_ex[x].metabolites.keys())[0].id for x in range(len(pepe_ex))]

ex_kegg_ids = []
ex_ids = []
ex_df = []
for i,rec in enumerate(carey_ex):
    try:
        met = list(rec.metabolites.keys())[0]
        kegg = met.annotation['kegg.compound']
    except:
        kegg=''
    ex_kegg_ids.extend(kegg)
    ex_ids.extend([i for _ in range(len(kegg))])
    ex_df.append(kegg)
#carey_ex_mets = [.name for x in range(len(carey_ex))]

In [168]:
inter, ids_l, ids_i = np.intersect1d(np.asarray(ex_kegg_ids), np.asarray(pepe_ex_mets), return_indices=True)
inter

array(['C00001', 'C00007', 'C00009', 'C00011', 'C00014', 'C00019',
       'C00022', 'C00025', 'C00026', 'C00027', 'C00031', 'C00033',
       'C00037', 'C00041', 'C00042', 'C00047', 'C00049', 'C00051',
       'C00058', 'C00059', 'C00060', 'C00062', 'C00064', 'C00065',
       'C00073', 'C00077', 'C00078', 'C00079', 'C00080', 'C00082',
       'C00084', 'C00086', 'C00088', 'C00095', 'C00097', 'C00114',
       'C00116', 'C00122', 'C00123', 'C00127', 'C00134', 'C00135',
       'C00137', 'C00143', 'C00147', 'C00148', 'C00149', 'C00152',
       'C00153', 'C00155', 'C00158', 'C00159', 'C00183', 'C00186',
       'C00187', 'C00188', 'C00189', 'C00212', 'C00242', 'C00249',
       'C00250', 'C00253', 'C00255', 'C00256', 'C00262', 'C00288',
       'C00294', 'C00302', 'C00314', 'C00315', 'C00329', 'C00330',
       'C00334', 'C00385', 'C00387', 'C00407', 'C00504', 'C00534',
       'C00559', 'C00568', 'C00712', 'C00864', 'C01279', 'C01300',
       'C01528', 'C01530', 'C05512', 'C14818'], dtype='<U6')

In [185]:
df_ex_pepe['Name']=[df_ex_pepe.Metabolites.iloc[x].name for x in range(len(df_ex_pepe))]
df_ex_pepe.iloc[np.isin(np.arange(len(pepe_ex_ids)), ids_i)]

,Reaction,Metabolites,Bounds,Name
0,EXC_BOTH_C00001_e: C00001_e <=>,C00001_e,"(-50.0, 50.0)",H2O
1,EXC_BOTH_C00007_e: C00007_e <=>,C00007_e,"(-50.0, 50.0)",Oxygen
2,EXC_BOTH_C00009_e: C00009_e <=>,C00009_e,"(-50.0, 50.0)",Orthophosphate
3,EXC_BOTH_C00011_e: C00011_e <=>,C00011_e,"(-50.0, 50.0)",CO2
5,EXC_BOTH_C00014_e: C00014_e <=>,C00014_e,"(-50.0, 50.0)",NH3
6,EXC_BOTH_C00019_e: C00019_e <=>,C00019_e,"(-50.0, 50.0)",S-Adenosyl-L-methionine
8,EXC_BOTH_C00022_e: C00022_e <=>,C00022_e,"(-50.0, 50.0)",Pyruvate
9,EXC_BOTH_C00025_e: C00025_e <=>,C00025_e,"(-50.0, 50.0)",L-Glutamate
10,EXC_BOTH_C00026_e: C00026_e <=>,C00026_e,"(-50.0, 50.0)",2-Oxoglutarate
11,EXC_BOTH_C00027_e: C00027_e <=>,C00027_e,"(-50.0, 50.0)",Hydrogen peroxide


In [188]:
not_included = np.asarray(ex_ids)[~np.isin(np.arange(len(ex_ids)), ids_l)]
included = np.asarray(ex_ids)[np.isin(np.arange(len(ex_ids)), ids_l)]

df_ex_carey['KEGG']=ex_df
print(len(included))
print(len(not_included))
print(len(df_ex_carey))
df_ex_carey#.iloc[included]

88
59
195


,Reaction,Metabolites,Bounds,KEGG
0,EX_4abut_e: 4abut_e <=>,4abut_e,"(-1000.0, 1000.0)",[C00334]
1,EX_ac_e: ac_e <=>,ac_e,"(-1000.0, 1000.0)",[C00033]
2,EX_acald_e: acald_e <=>,acald_e,"(-1000.0, 1000.0)",[C00084]
3,EX_ade_e: ade_e <=>,ade_e,"(-1000.0, 1000.0)",[C00147]
4,EX_adn_e: adn_e <=>,adn_e,"(-1000.0, 1000.0)",[C00212]
5,EX_akg_e: akg_e <=>,akg_e,"(-1000.0, 1000.0)",[C00026]
6,EX_ala__L_e: ala__L_e <=>,ala__L_e,"(-1000.0, 1000.0)","[C00041, C01401]"
7,EX_arg__L_e: arg__L_e <=>,arg__L_e,"(-1000.0, 1000.0)","[C02385, C00062]"
8,EX_asn__L_e: asn__L_e <=>,asn__L_e,"(-1000.0, 1000.0)","[C16438, C00152]"
9,EX_asp__L_e: asp__L_e <=>,asp__L_e,"(-1000.0, 1000.0)","[C00049, C16433]"
